In [1]:
#@title Import Libraries & Set Options

import pandas as pd
pd.set_option('display.max_columns', None)

import matplotlib.pyplot as plt

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

#from google.colab import drive
#drive.mount('/content/drive')

import warnings
warnings.filterwarnings("ignore")

In [2]:
#@title Declare Data Path

data_path = 'Mapping Tables.xlsx'

In [3]:
#@title Load Data To Model

# Load the Excel files
data = pd.read_excel(data_path, sheet_name=None)

# Access the individual sheets as needed
Instance_df = data['Instance-ID']
Zone_df = data['Zone-ID']
Class_df = data['Class-ID']
Race_df = data['Race-ID']
NPC_df = data['NPC-ID']
NPC_Data_df = data['NPC-Data']
Entries_df = data['Entries_Clean']

In [4]:
#@title Format Collected Entries to Dataframe

pivot_df = Entries_df.pivot(index=['Server', 'Character Name', 'Character ID'], columns='Field', values='Field Value')
pivot_df.reset_index(inplace=True)
#pivot_df.head()

sub_df = pivot_df[['Server','Character Name','Character ID','class_id','date','guild','last_words','level','map_id','map_pos','race_id','source_id']]
#sub_df.head()

In [5]:
#@title Check: Identify Missing Records

nan_counts = sub_df.isna().sum()
#nan_counts

In [6]:
#@title Drop NaN Records

sub_df_dropna = sub_df.dropna()

In [7]:
#@title Merge Dataframes

# Map zone IDs to zone names
zone_map = Zone_df.set_index('Zone ID')['Zone'].to_dict()
sub_df_dropna['map_name'] = sub_df_dropna['map_id'].apply(lambda x: zone_map.get(int(x), x))

# Map race IDs to race names
race_map = Race_df.set_index('Race ID')['Race'].to_dict()
sub_df_dropna['race_name'] = sub_df_dropna['race_id'].apply(lambda x: race_map.get(int(x), x))

# Map class IDs to class names
class_map = Class_df.set_index('Class ID')['Class'].to_dict()
sub_df_dropna['class_name'] = sub_df_dropna['class_id'].apply(lambda x: class_map.get(int(x), x))

# Map source IDs to NPC names
npc_map = NPC_df.set_index('NPC ID')['NPC'].to_dict()
sub_df_dropna['npc_name'] = sub_df_dropna['source_id'].apply(lambda x: npc_map.get(int(x), ""))

# Map source IDs to NPC data fields
npc_data_map_name = NPC_Data_df.set_index('ID')['Name'].to_dict()
npc_data_map_start_level = NPC_Data_df.set_index('ID')['Start Level'].to_dict()
npc_data_map_end_level = NPC_Data_df.set_index('ID')['End Level'].to_dict()
npc_data_map_elite = NPC_Data_df.set_index('ID')['Elite'].to_dict()
npc_data_map_rare = NPC_Data_df.set_index('ID')['Rare'].to_dict()
npc_data_map_boss = NPC_Data_df.set_index('ID')['Boss'].to_dict()
npc_data_map_type = NPC_Data_df.set_index('ID')['Type'].to_dict()

sub_df_dropna['npc_name'] = sub_df_dropna['source_id'].apply(lambda x: npc_data_map_name.get(int(x), ""))
sub_df_dropna['npc_start_level'] = sub_df_dropna['source_id'].apply(lambda x: npc_data_map_start_level.get(int(x), ""))
sub_df_dropna['npc_end_level'] = sub_df_dropna['source_id'].apply(lambda x: npc_data_map_end_level.get(int(x), ""))
sub_df_dropna['npc_elite_status'] = sub_df_dropna['source_id'].apply(lambda x: npc_data_map_elite.get(int(x), ""))
sub_df_dropna['npc_rare_status'] = sub_df_dropna['source_id'].apply(lambda x: npc_data_map_rare.get(int(x), ""))
sub_df_dropna['npc_boss_status'] = sub_df_dropna['source_id'].apply(lambda x: npc_data_map_boss.get(int(x), ""))
sub_df_dropna['npc_type'] = sub_df_dropna['source_id'].apply(lambda x: npc_data_map_type.get(int(x), ""))

In [8]:
#@title Perform Data Cleaning

sub_df_dropna['level'] = pd.to_numeric(sub_df_dropna['level'], errors='coerce')

sub_df_dropna['level_range'] = pd.np.where(sub_df_dropna['level'] <= 9, '1-9',
                                           pd.np.where(sub_df_dropna['level'] <= 19, '10-19',
                                                       pd.np.where(sub_df_dropna['level'] <= 29, '20-29',
                                                                   pd.np.where(sub_df_dropna['level'] <= 39, '30-39',
                                                                               pd.np.where(sub_df_dropna['level'] <= 49, '40-49',
                                                                                           pd.np.where(sub_df_dropna['level'] <= 59, '50-59', '60')
                                                                                          )
                                                                              )
                                                                  )
                                                      )
                                          )

sub_df_dropna['npc_start_level'] = pd.to_numeric(sub_df_dropna['npc_start_level'], errors='coerce')
sub_df_dropna['npc_end_level'] = pd.to_numeric(sub_df_dropna['npc_end_level'], errors='coerce')
sub_df_dropna['npc_avg_level'] = round((sub_df_dropna['npc_start_level'] + sub_df_dropna['npc_end_level']) / 2)
sub_df_dropna['npc_less_player_level'] = sub_df_dropna['npc_avg_level'] - sub_df_dropna['level']

sub_df_dropna['date'] = pd.to_datetime(sub_df_dropna['date'], unit='s')
sub_df_dropna['map_pos'] = sub_df_dropna['map_pos'].str.strip('"')  # Remove double quotes
sub_df_dropna[['x', 'y']] = sub_df_dropna['map_pos'].str.split(',', expand=True).astype(float)
sub_df_dropna['last_words'] = sub_df_dropna['last_words'].str.replace('"', '')
sub_df_dropna['guild'] = sub_df_dropna['guild'].str.replace('"', '')

In [9]:
#@title Identify Missing Records

nan_counts = sub_df_dropna.isna().sum()
#nan_counts

In [10]:
sub_df_dropna.to_csv('clean_data.csv', index=False)

In [11]:
sub_df_dropna

Field,Server,Character Name,Character ID,class_id,date,guild,last_words,level,map_id,map_pos,race_id,source_id,map_name,race_name,class_name,npc_name,npc_start_level,npc_end_level,npc_elite_status,npc_rare_status,npc_boss_status,npc_type,level_range,npc_avg_level,npc_less_player_level,x,y
0,Bloodsail Buccaneers,Abolishorhc,2227,4,2023-05-12 11:46:16,HC Runaways,hi,8,1429,"0.6866,0.7768",1,330,Elwynn Forest,Human,Rogue,Princess,9.0,9.0,0.0,0.0,0.0,Beast,1-9,9.0,1.0,0.6866,0.7768
1,Bloodsail Buccaneers,Ajayciaone,46741,1,2023-05-11 20:14:46,HC Doobsters,you not gonn ahelp?,9,1426,"0.7327,0.5164",1,1115,Dun Morogh,Human,Warrior,Rockjaw Skullthumper,8.0,9.0,0.0,0.0,0.0,Humanoid,1-9,8.0,-1.0,0.7327,0.5164
2,Bloodsail Buccaneers,Alathria,53702,4,2023-05-12 00:19:53,HC Doobsters,ill take one,9,1438,"0.4463,0.5996",4,2011,Teldrassil,Night Elf,Rogue,Gnarlpine Augur,8.0,9.0,0.0,0.0,0.0,Humanoid,1-9,8.0,-1.0,0.4463,0.5996
3,Bloodsail Buccaneers,Aldorus,32139,1,2023-05-12 10:13:14,HC Relax n Chill,the karate kid died,17,1439,"0.5274,0.3442",3,10373,Darkshore,Dwarf,Warrior,Xabraxxis,19.0,19.0,0.0,0.0,0.0,Demon,10-19,19.0,2.0,0.5274,0.3442
4,Bloodsail Buccaneers,Alkaia,41342,11,2023-05-12 10:37:07,HC Pwnda Posse,good morning,14,1439,"0.3869,0.5355",4,2167,Darkshore,Night Elf,Druid,Blackwood Pathfinder,12.0,13.0,0.0,0.0,0.0,Humanoid,10-19,12.0,-2.0,0.3869,0.5355
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45704,legacy,ßuckshothc,61523,3,2023-05-09 21:00:58,HC Cuties,lol,7,1438,"0.5363,0.4904",4,2038,Teldrassil,Night Elf,Hunter,Lord Melenas,8.0,8.0,0.0,0.0,0.0,Demon,1-9,8.0,1.0,0.5363,0.4904
45705,legacy,ßuffalotrace,5853,3,2023-04-30 19:00:02,HC Chad Gang,"Our brave brother, ßuffalotrace the Hunter, ha...",20,1433,"0.2918,0.5369",3,7040,Redridge Mountains,Dwarf,Hunter,Black Dragonspawn,52.0,53.0,1.0,0.0,0.0,Dragonkin,20-29,52.0,32.0,0.2918,0.5369
45709,legacy,ßæ,7819,4,2023-05-15 02:00:37,HC Hades,Ooo |cff1eff00|Hitem:6556::::::584:1118007808:...,15,1437,"0.5612,0.6457",1,1111,Wetlands,Human,Rogue,Leech Stalker,21.0,22.0,0.0,0.0,0.0,Beast,10-19,22.0,7.0,0.5612,0.6457
45710,legacy,ßôrn,54128,1,2023-04-06 18:00:45,HC Fallen Empire,anybody got a good website to watch movies,12,1432,"0.3028,0.3141",3,1190,Loch Modan,Dwarf,Warrior,Mountain Boar,10.0,11.0,0.0,0.0,0.0,Beast,10-19,10.0,-2.0,0.3028,0.3141
